In [124]:
# Import base dependencies
import pandas as pd
import requests
import time
from datetime import datetime
import math
from config import av_api_key as api_key
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression

# Import ML dependencies
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import SimpleRNN, Flatten, TimeDistributed, LSTM

In [125]:
# Import raw data from platform export
# IRA funds
ira_funds = [{'symbol':"AGX", 'basis':93.089}, 
             {'symbol':"APP", 'basis':276.123}, 
             {'symbol':"ARQT", 'basis':15.932}, 
             {'symbol':"ATGE"}, {'symbol':"BRK-B"}, {'symbol':"BLBD"}, {'symbol':"CAAP"}, 
             {'symbol':"CCL", 'basis':17.999}, 
             {'symbol':"CLS", 'basis':69.342}, 
             {'symbol':"CRDO", 'basis': 36.592}, 
             {'symbol':"EAT", 'basis':126.365}, 
             {'symbol':"EZPW", 'basis': 15.46}, 
             {'symbol':"MFC"}, 
             {'symbol':"PPC", 'basis': 45.45}, 
             {'symbol':"STRL"}, 
             {'symbol':"SKYW", 'basis':83.177}, 
             {'symbol':"TWLO", 'basis':100.519}, 
             {'symbol':"UBER", 'basis':54.777}, 
             {'symbol':"WFC", 'basis':66.113}
             ]

# Brokerage
brokerage_funds = [{'symbol':"AGX", 'basis':112.333}, 
                   {'symbol':"APP", 'basis':262.705}, 
                   {'symbol':"ARQT", 'basis':15.996}, 
                   {'symbol':"ATGE", 'basis':97.431}, 
                   {'symbol':"CCL", 'basis':20.829}, 
                   {'symbol':"CLS", 'basis':79.030}, 
                   {'symbol':"CRDO", 'basis':41.462}, 
                   {'symbol':"EAT", 'basis':135.74}, 
                   {'symbol':"EZPW", 'basis':18.187}, 
                   {'symbol':"FBTC", 'basis':75.244, 'is_etf': True}, 
                   {'symbol':"NVDA", 'basis':99.373}, 
                   {'symbol':"SKYW", 'basis':84.188}, 
                   {'symbol':"TWLO", 'basis':93.160}, 
                   {'symbol':"WFC", 'basis':73.186}
                   ]

In [139]:
def get_history(symbol, api_key, days=252):  # ~1 year default
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}&outputsize=full&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if "Time Series (Daily)" not in response:
            error_msg = response.get('Note', response.get('Information', 'Unknown error'))
            print(f"Error fetching price data for {symbol}: {error_msg}")
            print(f"Response keys: {list(response.keys())}")
            return None
        
        time_series = response["Time Series (Daily)"]
        df = pd.DataFrame.from_dict(time_series, orient="index", dtype=float)
        
        # Debug: Print available columns
        # print(f"Columns for {symbol}: {list(df.columns)}")
        
        # Rename columns dynamically
        column_map = {
            col: name for col, name in [
                ("1. open", "Open"), ("2. high", "High"), ("3. low", "Low"),
                ("4. close", "Close"), ("5. volume", "Volume"), ("6. volume", "Volume"),
                ("7. adjusted close", "Adjusted Close"), ("8. dividend amount", "Dividend")
            ] if col in df.columns
        }
        if "5. volume" not in df.columns and "6. volume" not in df.columns:
            print(f"No volume data for {symbol}")
            return None
        
        df = df.rename(columns=column_map)
        df.index = pd.to_datetime(df.index)
        df = df.sort_index().tail(days)
        return df
    except Exception as e:
        print(f"Exception fetching price data for {symbol}: {str(e)}")
        return None

def get_fundamentals(symbol, api_key, current_price):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if not response or "Symbol" not in response:
            error_msg = response.get('Note', response.get('Information', 'No data'))
            print(f"Error fetching fundamentals for {symbol}: {error_msg}")
            print(f"Full response: {response}")
            return None
        
        def safe_float(value, default):
            if value in [None, 'None', '']:
                return default
            try:
                return float(value)
            except (ValueError, TypeError):
                return default
        
        pe_ratio = safe_float(response.get('PERatio'), float('inf'))
        pb_ratio = safe_float(response.get('PriceToBookRatio'), float('inf'))
        
        # Calculate EPS and Book Value
        eps = current_price / pe_ratio if pe_ratio != float('inf') and pe_ratio != 0 else 0
        book_value = current_price / pb_ratio if pb_ratio != float('inf') and pb_ratio != 0 else 0
        
        fundamentals = {
            'pe_ratio': pe_ratio,
            'pb_ratio': pb_ratio,
            'dividend_yield': safe_float(response.get('DividendYield'), 0),
            'debt_to_equity': safe_float(response.get('DebtToEquityRatio'), float('inf')),
            'eps': eps,
            'book_value': book_value
        }
        
        return fundamentals
    except Exception as e:
        print(f"Exception fetching fundamentals for {symbol}: {str(e)}")
        return None

def calculate_vwap(df, days=126):  # ~2 quarters
    if 'Volume' not in df.columns:
        print("Missing Volume column in DataFrame")
        return None
    
    vwap_analysis = df[-days:].copy()
    vwap_analysis['Cumulative_LTPV'] = (vwap_analysis['Low'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_HTPV'] = (vwap_analysis['High'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_Volume'] = vwap_analysis['Volume'].cumsum()
    vwap_analysis['Entry'] = round(vwap_analysis['Cumulative_LTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    vwap_analysis['Exit'] = round(vwap_analysis['Cumulative_HTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    return vwap_analysis[-1:].copy()

def build_analysis_table(ticker_symbols, api_key, margin_of_safety=0.9, vwap_days=126, graham_margin=0.9):
    portfolio = []
    
    for ticker in ticker_symbols:
        symbol = ticker['symbol']
        is_etf = ticker.get('is_etf', False)
        
        # Get price data
        raw_data = get_history(symbol, api_key)
        if raw_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Get fundamentals (skip for ETFs)
        current_price = raw_data['Close'].iloc[-1]  # Use Close for fundamental calcs
        fundamentals = None if is_etf else get_fundamentals(symbol, api_key, current_price)
        if not is_etf and fundamentals is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Calculate VWAP
        vwap_data = calculate_vwap(raw_data, days=vwap_days)
        if vwap_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Extract data
        basis = ticker.get('basis', 0)
        market_price = round(raw_data['Low'].iloc[-1], 2)  # Use Low for buys
        entry_price = round(vwap_data['Entry'].iloc[0], 2)
        exit_price = round(vwap_data['Exit'].iloc[0], 2)
        buy_threshold = round(entry_price * margin_of_safety, 2)  # 10% margin
        
        # Graham buy threshold (for stocks only)
        graham_buy_threshold = None
        if not is_etf:
            if fundamentals['eps'] > 0 and fundamentals['book_value'] > 0:
                # Calculate desired price where P/E × P/B = 36
                desired_price = math.sqrt(27 * fundamentals['eps'] * fundamentals['book_value'])
                graham_buy_threshold = round(desired_price * graham_margin, 2)  # 5% margin
            else:
                graham_buy_threshold = buy_threshold  # Default to VWAP threshold
        
        # Volume filter: 20% of 21-day average
        avg_volume = raw_data['Volume'][-21:].mean()
        today_volume = raw_data['Volume'].iloc[-1]
        volume_ok = today_volume >= avg_volume * 0.2
        
        # Graham's fundamental checks (for stocks only)
        graham_ok = True
        if not is_etf:
            graham_ok = (
                (fundamentals['pe_ratio'] < 18 and fundamentals['pb_ratio'] < 1.5) or
                (fundamentals['pe_ratio'] * fundamentals['pb_ratio'] < 27 and 
                 fundamentals['pe_ratio'] < 100 and fundamentals['pb_ratio'] < 10)
                 ) and fundamentals['dividend_yield'] >= 0 and fundamentals['debt_to_equity'] < 2
        
        # Decision logic
        decision = "Hold"
        if market_price <= min(buy_threshold, graham_buy_threshold or float('inf')) and volume_ok and graham_ok:
            decision = "Buy"
        elif market_price >= exit_price and volume_ok:
            decision = "Sell"
        
        # Prepare fundamentals for output
        pe_ratio = None if is_etf else fundamentals['pe_ratio']
        pb_ratio = None if is_etf else fundamentals['pb_ratio']
        dividend_yield = None if is_etf else fundamentals['dividend_yield']
        
        portfolio.append([
            symbol, basis, market_price, 
            buy_threshold, graham_buy_threshold, exit_price,
            pe_ratio, pb_ratio, dividend_yield, decision
        ])
        
        # Minimal delay for server stability (75 calls/minute = ~0.8 seconds/call)
        time.sleep(0.1)
    
    return portfolio

In [132]:
b_portfolio = build_analysis_table(brokerage_funds, api_key, margin_of_safety=0.97, vwap_days=126)
brokerage_df = pd.DataFrame(b_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [133]:
brokerage_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,AGX,112.333,160.27,133.42,74.28,147.63,26.59,5.940,0.0092,Sell
1,APP,262.705,286.85,287.43,21.75,325.81,68.05,95.500,0.0000,Hold
2,ARQT,15.996,14.74,12.40,12.40,13.98,inf,11.530,0.0000,Sell
3,ATGE,97.431,108.31,91.97,87.00,98.79,20.23,2.777,0.0000,Sell
4,CCL,20.829,19.02,21.45,19.08,23.14,12.63,2.706,0.0000,Hold
5,CLS,79.030,91.22,89.80,40.74,100.29,24.92,6.950,0.0000,Hold
6,CRDO,41.462,46.74,52.23,1.88,59.09,1610.57,13.260,0.0000,Hold
7,EAT,135.740,130.31,134.94,36.72,146.52,18.51,23.050,0.0000,Hold
8,EZPW,18.187,15.29,13.61,23.92,14.58,13.15,1.058,0.0000,Sell
9,FBTC,75.244,84.47,75.57,NaN,81.09,NaN,NaN,NaN,Sell


In [140]:
r_portfolio = build_analysis_table(ira_funds, api_key, margin_of_safety=0.9, vwap_days=126)
retirement_df = pd.DataFrame(r_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [141]:
retirement_df[retirement_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,AGX,93.089,160.27,123.80,60.94,147.63,26.59,5.940,0.0092,Sell
1,APP,276.123,286.85,266.69,17.84,325.81,68.05,95.500,0.0000,Hold
2,ARQT,15.932,14.74,11.50,11.50,13.98,inf,11.530,0.0000,Sell
7,CCL,17.999,19.02,19.90,15.65,23.14,12.63,2.706,0.0000,Hold
8,CLS,69.342,91.22,83.32,33.43,100.29,24.92,6.950,0.0000,Hold
9,CRDO,36.592,46.74,48.46,1.54,59.09,1610.57,13.260,0.0000,Hold
10,EAT,126.365,130.31,125.20,30.13,146.52,18.51,23.050,0.0000,Hold
11,EZPW,15.460,15.29,12.63,19.62,14.58,13.15,1.058,0.0000,Sell
13,PPC,45.450,45.37,44.49,38.32,51.42,9.08,3.492,0.0000,Hold
15,SKYW,83.177,91.96,87.82,108.13,101.85,10.73,1.534,0.0000,Hold


In [143]:
retirement_df[retirement_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
3,ATGE,0.0,108.31,85.33,71.38,98.79,20.23,2.777,0.000,Sell
4,BRK-B,0.0,535.26,434.12,521.07,493.19,13.09,1.793,0.000,Sell
5,BLBD,0.0,36.30,33.33,21.14,39.14,11.16,6.320,0.000,Hold
6,CAAP,0.0,19.27,16.41,18.04,19.05,11.15,2.296,0.000,Sell
12,MFC,0.0,31.34,26.96,29.53,30.75,15.33,1.631,0.039,Sell
14,STRL,0.0,161.20,125.07,73.01,148.18,18.07,6.230,0.000,Sell


In [144]:
# Research new investments
if 1==1:
    custom_watchlist = [{'symbol':'EZPW'}, 
                        {'symbol':'QTWO'}, 
                        {'symbol':'PPC'}, 
                        {'symbol':'ATGE'}, 
                        {'symbol':'BRK-B'}, 
                        {'symbol':'TWLO'}, 
                        {'symbol':'MFC'}, 
                        {'symbol':'STRL'}, 
                        {'symbol':'GRBK'}, 
                        {'symbol':'UBER'}, 
                        {'symbol':'BLBD'}, 
                        {'symbol':'CAAP'}, 
                        {'symbol':'AMZN'}, 
                        {'symbol':'GOOG'}, 
                        {'symbol':'GOOGL'}, 
                        {'symbol':'GME'}, 
                        ] 
    watch_list = build_analysis_table(custom_watchlist, api_key, vwap_days=63)
else:
    symbol_input = input("Input symbol to research: ").upper()
    one_off_lookup = [{'symbol': symbol_input}]
    watch_list = build_analysis_table(one_off_lookup)

# Create watch list data frame
watch_list_df = pd.DataFrame(watch_list, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [145]:
watch_list

[['EZPW', 0, 15.29, 13.28, 19.62, 15.38, 13.15, 1.058, 0, 'Hold'],
 ['QTWO', 0, 79.59, 71.32, 71.32, 82.95, inf, 9.63, 0, 'Hold'],
 ['PPC', 0, 45.37, 45.38, 38.32, 52.7, 9.08, 3.492, 0.0, 'Hold'],
 ['ATGE', 0, 108.31, 89.44, 71.38, 103.94, 20.23, 2.777, 0, 'Sell'],
 ['BRK-B', 0, 535.26, 451.17, 521.07, 515.29, 13.09, 1.793, 0, 'Sell'],
 ['TWLO', 0, 97.13, 96.26, 96.26, 112.52, inf, 1.879, 0, 'Hold'],
 ['MFC', 0, 31.34, 26.25, 29.53, 30.16, 15.33, 1.631, 0.039, 'Sell'],
 ['STRL', 0, 161.2, 111.96, 73.01, 132.95, 18.07, 6.23, 0, 'Sell'],
 ['GRBK', 0, 59.1, 51.48, 81.61, 59.59, 7.25, 1.642, 0, 'Hold'],
 ['UBER', 0, 81.8, 64.53, 32.06, 75.24, 18.48, 8.18, 0, 'Sell'],
 ['BLBD', 0, 36.3, 30.44, 21.14, 35.6, 11.16, 6.32, 0, 'Sell'],
 ['CAAP', 0, 19.27, 16.03, 18.04, 18.69, 11.15, 2.296, 0, 'Sell'],
 ['AMZN', 0, 186.4, 173.39, 62.22, 200.61, 30.94, 6.59, 0, 'Hold'],
 ['GOOG', 0, 163.66, 148.73, 74.6, 170.63, 18.53, 5.83, 0.0051, 'Hold'],
 ['GOOGL', 0, 161.87, 147.04, 75.89, 168.7, 18.31, 5.58,

In [146]:
watch_list_df[watch_list_df['rating']=='Buy']

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating


In [147]:
watch_list_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,EZPW,0,15.29,13.28,19.62,15.38,13.15,1.058,0.0000,Hold
1,QTWO,0,79.59,71.32,71.32,82.95,inf,9.630,0.0000,Hold
2,PPC,0,45.37,45.38,38.32,52.70,9.08,3.492,0.0000,Hold
3,ATGE,0,108.31,89.44,71.38,103.94,20.23,2.777,0.0000,Sell
4,BRK-B,0,535.26,451.17,521.07,515.29,13.09,1.793,0.0000,Sell
5,TWLO,0,97.13,96.26,96.26,112.52,inf,1.879,0.0000,Hold
6,MFC,0,31.34,26.25,29.53,30.16,15.33,1.631,0.0390,Sell
7,STRL,0,161.20,111.96,73.01,132.95,18.07,6.230,0.0000,Sell
8,GRBK,0,59.10,51.48,81.61,59.59,7.25,1.642,0.0000,Hold
9,UBER,0,81.80,64.53,32.06,75.24,18.48,8.180,0.0000,Sell


In [65]:
# Machine Learning

In [66]:
# Data processing and clearning
# Must be in numpy array or tf.Dataset object format

In [67]:
# Feature selection and normalization

In [68]:
# Build model

In [69]:
# Train model

In [70]:
# Evaluate model

In [71]:
# Refine model through hyperparameter tuning